In [1]:
import glob
import json
import re
import os
from os import path


In [2]:
def clean_images(basepath):
    count = 0
    images = glob.glob(f'{basepath}/**/*.jpg')
    for image in images:
        os.remove(image)
        count += 1
    print("Cleaned ", count, " images.")


In [3]:
clean_images("train")
clean_images("test")
clean_images("validation")

Cleaned  252  images.
Cleaned  108  images.
Cleaned  132  images.


In [4]:
def process_movie(basepath, moviePath, label, split):
    if path.exists(f'{basepath}/{moviePath}'):
        print("Processing movie, ", moviePath, " with label, ", label, " and split, ", split)
        movie = re.sub(r'(\.mp4$)', '', moviePath)
        !ffmpeg -i {basepath}/{moviePath} -r 1/1 {split}/{label}/{movie}%08d.jpg
        return 1
    return 0

In [5]:
#ext_drive = 'G:/deepfake/**/'
#metadataFiles = glob.glob('G:/deepfake/**/**/metadata.json')
metadataFiles = glob.glob('movies/**/metadata.json')
MAX_MOVIES=10

moviesProcessed = 0
testSamplingRate = 5  # every 5th movie, put into test, every sixth movie put into validation

def processMetadataFile(metadataFile, moviesProcessed):
    numRealMovies = 0
    numFakeMovies = 0
    originalsProcessed = 0

    for metadata in metadataFile:
        print("found metadata, ", metadata)
        basepath=re.sub(r'(metadata.json$)', '', metadata)
        with open(metadata) as f:        
            data = json.load(f)
            for key in data:
                if moviesProcessed > MAX_MOVIES:
                    print("moviesProcessed > MAX")
                    break
                label = data[key]['label']
                if moviesProcessed % testSamplingRate == 0:
                    split = "test"
                elif moviesProcessed % testSamplingRate == 1:
                    split = "validation"
                else:
                    split = "train"
                if label == "FAKE" and 2 * numFakeMovies > numRealMovies:
                    print("Skipping because we need balanced fake and real videos")
                elif label == "FAKE":
                    original = data[key]['original']
                    processed = process_movie(basepath, key, label, split)
                    moviesProcessed += processed
                    numFakeMovies += processed

                    processed = process_movie(basepath, original, "REAL", split)
                    moviesProcessed += processed
                    numRealMovies += processed
                    originalsProcessed += processed

                else:
                    numRealMovies += 1
                    processed = process_movie(basepath, key, label, split)

                    moviesProcessed += processed
    print("Processed originals", originalsProcessed, " videos for train.")
    return moviesProcessed
      
moviesProcessed += processMetadataFile(metadataFiles, moviesProcessed)
    



found metadata,  G:/deepfake\dfdc_train_part_01\dfdc_train_part_1\metadata.json
Processing movie,  zumqqvixhu.mp4  with label,  FAKE  and split,  test
Processing movie,  hntguogkqd.mp4  with label,  REAL  and split,  test


ffmpeg version git-2020-02-13-6d37ca8 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt --enable-amf
  libavutil      56. 41.100 

Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Skipping because we need balanced fake and real videos
Processing movie,  ooafcxxfrs.mp4  with label,  REAL  and split, 

ffmpeg version git-2020-02-13-6d37ca8 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt --enable-amf
  libavutil      56. 41.100 

Processing movie,  gshruegrcb.mp4  with label,  FAKE  and split,  train


...

ffmpeg version git-2020-02-13-6d37ca8 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt --enable-amf
  libavutil      56. 41.100 

Processing movie,  ywauoonmlr.mp4  with label,  REAL  and split,  test


ffmpeg version git-2020-02-13-6d37ca8 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt --enable-amf
  libavutil      56. 41.100 

moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_02\dfdc_train_part_2\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_03\dfdc_train_part_3\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_04\dfdc_train_part_4\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_22\dfdc_train_part_22\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_23\dfdc_train_part_23\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_24\dfdc_train_part_24\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_25\dfdc_train_part_25\metadata.json
moviesProcessed > MAX
found metadata,  G:/deepfake\dfdc_train_part_26\dfdc_train_part_26\metadata.json
moviesProcessed > MAX
Processed originals 4  videos for train.


ffmpeg version git-2020-02-13-6d37ca8 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.2.1 (GCC) 20200122
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt --enable-amf
  libavutil      56. 41.100 

In [6]:
print("Processed ", moviesProcessed, " videos for train.")

Processed  12  videos for train.


In [7]:
from detect_face import crop_face_save_jpg

In [8]:
trainImages = glob.glob('train/**/*.jpg')


In [9]:
# Find the faces in each image, and crop it out
# If the face can't be detected, then we remove the image from dataset
for image in trainImages:
    crop_face_save_jpg(image)

train\fake\gshruegrcb00000001.jpg  did not have a face
train\fake\gshruegrcb00000002.jpg  did not have a face
train\fake\gshruegrcb00000003.jpg  did not have a face
train\fake\gshruegrcb00000004.jpg  did not have a face
train\fake\gshruegrcb00000005.jpg  did not have a face
train\fake\gshruegrcb00000006.jpg  did not have a face
train\fake\gshruegrcb00000007.jpg  did not have a face
train\fake\gshruegrcb00000008.jpg  did not have a face
train\fake\gshruegrcb00000009.jpg  did not have a face
train\real\fmhiujydwo00000001.jpg  did not have a face
train\real\fmhiujydwo00000004.jpg  did not have a face
train\real\fmhiujydwo00000005.jpg  did not have a face
train\real\fmhiujydwo00000006.jpg  did not have a face
train\real\fmhiujydwo00000007.jpg  did not have a face
train\real\fmhiujydwo00000008.jpg  did not have a face
train\real\fmhiujydwo00000009.jpg  did not have a face
train\real\fmhiujydwo00000011.jpg  did not have a face
train\real\fmhiujydwo00000012.jpg  did not have a face
train\real

In [10]:
testImages = glob.glob('test/**/*.jpg')
for image in testImages:
    crop_face_save_jpg(image)

test\fake\zumqqvixhu00000001.jpg  did not have a face
test\fake\zumqqvixhu00000002.jpg  did not have a face
test\fake\zumqqvixhu00000003.jpg  did not have a face
test\fake\zumqqvixhu00000005.jpg  did not have a face
test\fake\zumqqvixhu00000006.jpg  did not have a face
test\fake\zumqqvixhu00000007.jpg  did not have a face
test\fake\zumqqvixhu00000008.jpg  did not have a face
test\fake\zumqqvixhu00000009.jpg  did not have a face
test\fake\zumqqvixhu00000010.jpg  did not have a face
test\fake\zumqqvixhu00000011.jpg  did not have a face
test\fake\zumqqvixhu00000012.jpg  did not have a face
test\real\hntguogkqd00000001.jpg  did not have a face
test\real\hntguogkqd00000002.jpg  did not have a face
test\real\hntguogkqd00000003.jpg  did not have a face
test\real\hntguogkqd00000005.jpg  did not have a face
test\real\hntguogkqd00000006.jpg  did not have a face
test\real\hntguogkqd00000007.jpg  did not have a face
test\real\hntguogkqd00000008.jpg  did not have a face
test\real\hntguogkqd00000009

In [11]:
print("Finished preprocessing movies")

Finished preprocessing movies


In [12]:
import numpy as np
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model
import pandas as pd
from pickle import dump

def extract_features(filepath):
    # load model
    model = VGG16()
    # remove the output layer
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    features = np.zeros((1, 4096))
    filepath = os.path.join(filepath,'*.jpg')
    faces = glob.glob(filepath)
    for face in faces:
        # load an image from file
        image = load_img(face, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        #print(image.shape)
        #prepare the image for the VGG model
        image = preprocess_input(image)
        # get extracted features
        features = np.append(features,  model.predict(image), axis = 0)
        #print(features.shape)
        # save to file
        #dump(features, f)
    #save to pd dataframe
    return pd.DataFrame(features)
train_fake_df = extract_features('train/fake/')
train_real_df = extract_features('train/real/')
test_real_df = extract_features('test/real/')
test_fake_df = extract_features('test/fake/')
print(train_fake_df)

Using TensorFlow backend.


   0     1     2     3     4         5     6     7         8     9     ...  \
0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  0.000000   0.0  ...   
1   0.0   0.0   0.0   0.0   0.0  0.509220   0.0   0.0  1.201362   0.0  ...   
2   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  1.668910   0.0  ...   
3   0.0   0.0   0.0   0.0   0.0  0.526295   0.0   0.0  2.078585   0.0  ...   

   4086      4087  4088      4089      4090  4091  4092  4093  4094      4095  
0   0.0  0.000000   0.0  0.000000  0.000000   0.0   0.0   0.0   0.0  0.000000  
1   0.0  0.210604   0.0  0.549762  1.273698   0.0   0.0   0.0   0.0  0.273202  
2   0.0  0.570012   0.0  0.564549  1.078326   0.0   0.0   0.0   0.0  0.000000  
3   0.0  0.369404   0.0  0.048695  0.647373   0.0   0.0   0.0   0.0  1.663213  

[4 rows x 4096 columns]


In [17]:
# write extracted features to disk
train_fake_df.to_csv('train_fake.csv')
train_real_df.to_csv('train_real.csv')
test_fake_df.to_csv('test_fake.csv')
test_real_df.to_csv('test_real.csv')

In [18]:
#model = VGG16()

In [22]:
#model.layers.pop()
#model.layers.pop()
#model.layers.pop()

#model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     